<a href="https://colab.research.google.com/github/yenlung/AI-Demo/blob/master/%E3%80%90Demo06%E3%80%91%E7%94%A8_RAG_%E6%89%93%E9%80%A0%E5%BF%83%E9%9D%88%E8%99%95%E6%96%B9%E7%B1%A4%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 讀入需要的套件

這裡主要用 `LangChain`, 這可以說整合各式 LLM 功能的方便套件。

In [2]:
!pip install -U nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [3]:
!pip install langchain langchain-community openai faiss-cpu unstructured tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━

讀入正確的 `nltk` 所需資料。

In [4]:
import nltk

In [5]:
nltk.data.path.append("/root/nltk_data")
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

讀入一大票需要的函式。

In [10]:
import os
import random
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

### 2. 讀入範例資料

我們這裡用聖嚴法師的《真正的快樂》一書為範例, 當然其實可以用更多的資料, 直接放入相對的資料夾 (這裡是設在 `books` 之下) 即可。包括這本書都在[《法鼓全集》](https://ddc.shengyen.org/)之下, 請注意版權屬「法鼓文化」所有, 我們只是作為範例。

In [6]:
# 下載 books.zip 檔案
!wget -O books.zip https://github.com/yenlung/AI-Demo/raw/refs/heads/master/books.zip

# 解壓縮 books.zip 到 books 資料夾
!unzip -o books.zip

--2024-10-30 11:49:08--  https://github.com/yenlung/AI-Demo/raw/refs/heads/master/books.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yenlung/AI-Demo/refs/heads/master/books.zip [following]
--2024-10-30 11:49:08--  https://raw.githubusercontent.com/yenlung/AI-Demo/refs/heads/master/books.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63072 (62K) [application/zip]
Saving to: ‘books.zip’

books.zip           100%[===================>]  61.59K  --.-KB/s    in 0.03s   

2024-10-30 11:49:09 (2.29 MB/s) - ‘books.zip’ saved [63072/63072]

Archive:  books.zip
   creating: books/
  inflating: 

### 3. 設定 OpenAI 金鑰

In [7]:
from getpass import getpass

In [8]:
api_key = getpass("請輸入您的 OpenAI API key: ")

請輸入您的 OpenAI API key: ··········


In [11]:
os.environ["OPENAI_API_KEY"] = api_key

### 4. 建立向量資料庫

#### Step 1: 加載資料夾中的文件

In [14]:
loader = DirectoryLoader("books", glob="*.txt")  # 替換為你的資料夾路徑
documents = loader.load()

#### Step 2: 將文件分割成較小的片段

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

#### Step 3: 使用 OpenAI 的嵌入來將文件轉為向量嵌入

In [16]:
embeddings = OpenAIEmbeddings()

<ipython-input-16-73ad2f8e367a>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


#### Step 4: 使用 FAISS 建立向量資料庫

In [17]:
vector_store = FAISS.from_documents(split_docs, embeddings)

#### Step 5: 建立檢索器

In [18]:
retriever = vector_store.as_retriever()

### 5. 打造心靈處方籤機器人

#### 選定語言模型

In [19]:
llm = ChatOpenAI(model="gpt-4o")

<ipython-input-19-e8ae6cf3228a>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o")


#### 定義一些心靈處方籤

In [20]:
spiritual_prescriptions = [
    "感謝給我們機會，順境、逆境，皆是恩人。",
    "身心常放鬆，逢人面帶笑；放鬆能使我們身心健康，帶笑容易增進彼此友誼。",
    "識人識己識進退，時時身心平安；知福惜福多培福，處處廣結善緣。",
    "平常心就是最自在、最愉快的心。",
    "知道自己的缺點愈多，成長的速度愈快，對自己的信心也就愈堅定。"
]

#### 建立一個結合檢索與生成的 RAG 問答鏈

In [21]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

#### 定義真正的心靈處方籤主函式

注意最主要還是設計 `prompt` 的型式。

In [22]:
def answer_user_question(question):
    # Step 8a: 抽取一條隨機的心靈處方籤
    chosen_prescription = random.choice(spiritual_prescriptions)

    # Step 8b: 檢索資料夾中的相關內容
    retriever_result = qa_chain.run(question)

    print(f"你抽到的心靈處方籤: {chosen_prescription}")

    # Step 8c: 自訂 prompt，結合心靈處方籤、上下文和使用者問題
    prompt = f"""
    使用者抽到了一個心靈處方籤，它的內容是：{chosen_prescription}

    以下是我們從資料庫中檢索到的內容，這些內容來自書中的資料，並與使用者的問題相關：
    {retriever_result}

    請根據「心靈處方籤」中的訊息，結合書中的資料，用類似的語氣和觀念來回應使用者的問題：
    「{question}」
    """

    # 使用 HumanMessage 包裝 prompt 並生成回答
    final_response = llm.invoke([HumanMessage(content=prompt)])

    return final_response.content

In [23]:
# 使用範例：回答一個使用者問題
user_question = "我該如何面對現在的挑戰？"
response = answer_user_question(user_question)

print(response)

<ipython-input-22-fa93502390d4>:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever_result = qa_chain.run(question)


你抽到的心靈處方籤: 識人識己識進退，時時身心平安；知福惜福多培福，處處廣結善緣。
面對現在的挑戰，首先要識人識己識進退，保持冷靜和沉著。這樣能夠讓你在困境中找到合適的方向和策略。可以運用冥想、禱告或持誦安定的咒語等方法，讓自己心平氣和。如果你有信仰，不妨將困難交給信仰去承擔，這樣能減輕你的焦慮。

同時，知福惜福多培福，轉變觀念，看到事物積極的一面。無論結果如何，只要全力以赴地去面對，不必過分在意成敗得失。重點是從過程中獲得經驗和成長，這些都是無形的福氣。處處廣結善緣，保持樂觀和積極的心態，因為希望總是存在的。相信無論過程多麼艱難，只要心態正面，身心便能平安。
